In [ ]:
import os
import path_file
os.chdir(os.path.dirname(os.path.dirname(path_file.__file__)))
os.getcwd()

In [273]:
import sys
import cantera as ct
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [274]:
ultimate = [71.0, 6.0, 12.7, 1.3, 0.5, 0.0] # %m/m, dry basis
moistWB = 0.024 # fraction, wet basis
ashWB = 0.083 # fraction, wet basis

moistDB = moistWB / (1 - moistWB)
ashDB = ashWB / (1 - moistWB)
HHV_DB = 29.8

In [275]:
primaryMass = 0.00729 # kg/s
steamMass = 0.00184 # kg/s
coalPrimRatio = 0.910 # kg coal / kg primary

coalMass = primaryMass * coalPrimRatio
coalDryMass = coalMass * (1 - moistWB)

primaryO2 = 0.850
primaryAr = 0.126
primaryH2O = 0.024
primaryMW = primaryO2 * phases.Mw['O2'] + primaryAr * phases.Mw['Ar'] + primaryH2O * phases.Mw['H2O']
primaryMoles = primaryMass / primaryMW

secondaryMoles = steamMass / phases.Mw['H2O']

agent = phases.stream()
molesList = agent.species_moles
molesList[phases.indices['O2']] = primaryMoles * primaryO2
molesList[phases.indices['Ar']] = primaryMoles * primaryAr
molesList[phases.indices['H2O']] = primaryMoles * primaryH2O + secondaryMoles
agent.species_moles = molesList

fuel = fs.create_fuel_stream(coalDryMass, ultimate, ashDB, moistDB, HHV_DB)
print(fuel.get_mass())
print(agent.get_mass())


0.006633900000000001
0.00913


In [276]:
# outlet = gs.gasify_nonisot(fuel, agent, T0=298.15, P=101325, heatLossFraction=0.10, charFormation=0.10, directMethaneConv=0.005)
outlet = gs.gasify_isot(fuel, agent, T=1375, P=101325, charFormation=0.05, directMethaneConv=0.00)
print(outlet.T)

species = ['N2', 'Ar', 'H2O', 'CO', 'CO2', 'H2', 'CH4']
fracs = np.zeros(len(species))
for i, sp in enumerate(species):
    fracs[i] = outlet.get_syngas_fraction(sp, water=True, nitrogen=True)

print(sum(fracs))
impureFrac = fracs[0] + fracs[1] + fracs[2]
pureFracs = fracs[3:] / (1 - impureFrac)
print(sum(pureFracs))
print(list(pureFracs*100))
# for sp in phases.names_g:
#     print(f'{sp}: {outlet.species_moles[phases.indices[sp]] / sum(outlet.species_moles) * 100}')

1375.0
0.9998871557946895
0.9998591182995478
[51.039671945130294, 13.428747174539147, 35.51731881041433, 0.00017389987100444555]
